In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv('./.env')
os.environ.get('PINECONE_ENV')

'gcp-starter'

In [3]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

C:\Users\Jackson\workshop-llm-2\env_llm\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [7]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Carregando {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Carregando {file}')
        loader = Docx2txtLoader(file)
    else:
        print('Formato não suportado!')
        return None

    data = loader.load()
    return data

# Wikipedia Loader
def load_from_wikipedia(query, lang='pt', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [8]:
def chunk_data(data, chunk_size=1000):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

In [9]:
def embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_tokens / 1000 * 0.0001:.6f}')

In [10]:
data = load_document('docs/CLT.pdf')
chunks = chunk_data(data)

Carregando docs/CLT.pdf


In [11]:
embedding_cost(chunks)

Total de tokens: 252697
Custo de Embedding em USD: 0.025270


## insert embeddings

In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
vector = embeddings.embed_query(chunks[0].page_content)

In [14]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key="xxxx")

In [15]:
index_name = "linuxtips"

In [16]:
pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [17]:
vector_store = PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)

## Q&A

In [18]:
def get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    #from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(q)
    return answer

def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    #from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history

In [19]:
q = 'em que ano a clt foi criada?'
answer = get_answer(vector_store, q)
print(answer)

C:\Users\Jackson\workshop-llm-2\env_llm\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


A Consolidação das Leis do Trabalho (CLT) foi criada em 1943.


In [20]:
import time
i = 1
print('Digite sair para encerrar.')
while True:
    q = input(f'Pergunta: #{i}: ')
    i = i+1
    if q.lower() in ['sair']:
        print('Encerrando...')
        time.sleep(2)
        break

    answer = get_answer(vector_store, q)
    print(f'\nResposta: {answer}')
    print(f'\n {"-" * 50} \n')

Digite sair para encerrar.


Pergunta: #1:  Como funciona as férias ?



Resposta: As férias são um período de descanso remunerado concedido ao trabalhador após um período de trabalho contínuo. De acordo com a legislação trabalhista, a concessão das férias deve ser comunicada por escrito ao empregado com antecedência mínima de 30 dias. Durante as férias, o empregado não pode prestar serviços a outro empregador, a não ser que tenha um contrato de trabalho regularmente mantido. Se as férias forem concedidas após o prazo estabelecido, o empregador deve pagar em dobro a remuneração correspondente. Além disso, existem regras específicas para férias coletivas e para empregados menores de 18 anos.

 -------------------------------------------------- 



Pergunta: #2:  sair


Encerrando...
